Analysis of the SVD in Julia on a simple matrix.

In [1]:
A = [1 0 0 0 2
     0 0 3 0 0
     0 0 0 0 0
     0 4 0 0 0]

4×5 Array{Int64,2}:
 1  0  0  0  2
 0  0  3  0  0
 0  0  0  0  0
 0  4  0  0  0

Unlike the equivalent in Python, svd returns V not V' (V transpose)

In [2]:
U, σ, V = svd(A)
display(U), display(σ), display(V);

4×4 Array{Float64,2}:
 0.0  0.0  1.0   0.0
 0.0  1.0  0.0   0.0
 0.0  0.0  0.0  -1.0
 1.0  0.0  0.0   0.0

4-element Array{Float64,1}:
 4.0    
 3.0    
 2.23607
 0.0    

5×4 Array{Float64,2}:
 -0.0  -0.0  0.447214  0.0
  1.0   0.0  0.0       0.0
  0.0   1.0  0.0       0.0
 -0.0  -0.0  0.0       1.0
  0.0   0.0  0.894427  0.0

In [3]:
Σ = Diagonal(σ)

4×4 Diagonal{Float64}:
 4.0   ⋅    ⋅        ⋅ 
  ⋅   3.0   ⋅        ⋅ 
  ⋅    ⋅   2.23607   ⋅ 
  ⋅    ⋅    ⋅       0.0

Now re-create the matrix from the 3 elements returned from the SVD

In [4]:
A_2 = U * Σ * V'

4×5 Array{Float64,2}:
 1.0  0.0  0.0  0.0  2.0
 0.0  0.0  3.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  4.0  0.0  0.0  0.0

Now work out the pseudo inverse

In [5]:
A_inv = pinv(A)

5×4 Array{Float64,2}:
 0.2  0.0       0.0  0.0 
 0.0  0.0       0.0  0.25
 0.0  0.333333  0.0  0.0 
 0.0  0.0       0.0  0.0 
 0.4  0.0       0.0  0.0 

Now work out the pseudo inverse manually.
First we work out the inverse of Sigma, by inverting the non-zero elements.

In [6]:
Sigma_inv = zeros(Σ, Float64);
m, n = size(Σ)
for i = 1:m, j = 1:n
    if i==j
        if Σ[i,j] > 0.0
            Sigma_inv[i,j] = 1 / Σ[i,j]
        end
    end
end
display(Sigma_inv)

4×4 Diagonal{Float64}:
 0.25   ⋅         ⋅         ⋅ 
  ⋅    0.333333   ⋅         ⋅ 
  ⋅     ⋅        0.447214   ⋅ 
  ⋅     ⋅         ⋅        0.0

In [7]:
display(V)
display(Sigma_inv)
display(U')

5×4 Array{Float64,2}:
 -0.0  -0.0  0.447214  0.0
  1.0   0.0  0.0       0.0
  0.0   1.0  0.0       0.0
 -0.0  -0.0  0.0       1.0
  0.0   0.0  0.894427  0.0

4×4 Diagonal{Float64}:
 0.25   ⋅         ⋅         ⋅ 
  ⋅    0.333333   ⋅         ⋅ 
  ⋅     ⋅        0.447214   ⋅ 
  ⋅     ⋅         ⋅        0.0

4×4 Array{Float64,2}:
 0.0  0.0   0.0  1.0
 0.0  1.0   0.0  0.0
 1.0  0.0   0.0  0.0
 0.0  0.0  -1.0  0.0

In [8]:
A_inv_2 = V * Sigma_inv * U'

5×4 Array{Float64,2}:
 0.2  0.0       0.0  0.0 
 0.0  0.0       0.0  0.25
 0.0  0.333333  0.0  0.0 
 0.0  0.0       0.0  0.0 
 0.4  0.0       0.0  0.0 

In [9]:
test = [1/Σ[i,j] for i = 1:m, j = 1:n if Σ[i,j] > 0]

3-element Array{Float64,1}:
 0.25    
 0.333333
 0.447214

Now, try and find out the pseudo inverse via other means.

In [10]:
A_TxA = A'*A

5×5 Array{Int64,2}:
 1   0  0  0  2
 0  16  0  0  0
 0   0  9  0  0
 0   0  0  0  0
 2   0  0  0  4

In [11]:
#find eigen values and vectors on A'*A
lambda, V = eig(A_TxA)
display(lambda)
display(V)

5-element Array{Float64,1}:
  0.0        
  1.77636e-15
  5.0        
  9.0        
 16.0        

5×5 Array{Float64,2}:
  0.0  -0.894427  0.447214  0.0  0.0
  0.0   0.0       0.0       0.0  1.0
  0.0   0.0       0.0       1.0  0.0
 -1.0   0.0       0.0       0.0  0.0
  0.0   0.447214  0.894427  0.0  0.0

Now work out the inverse of eigenvalues, within tolerance and then create a diagonal matrix.
(A'*A)^-1*A'

In [12]:
tolerance = 1e-4
m=size(lambda)
lambda_inv = fill(0.0,m[1])
for i in 1:m[1]
    if lambda[i] > tolerance
        lambda_inv[i] = 1/lambda[i]
    end
end
lambda_inv_diag = Diagonal(lambda_inv)

5×5 Diagonal{Float64}:
 0.0   ⋅    ⋅    ⋅         ⋅    
  ⋅   0.0   ⋅    ⋅         ⋅    
  ⋅    ⋅   0.2   ⋅         ⋅    
  ⋅    ⋅    ⋅   0.111111   ⋅    
  ⋅    ⋅    ⋅    ⋅        0.0625

In [13]:
AA_inv = V * lambda_inv_diag * V'

5×5 Array{Float64,2}:
 0.04  0.0     0.0       0.0  0.08
 0.0   0.0625  0.0       0.0  0.0 
 0.0   0.0     0.111111  0.0  0.0 
 0.0   0.0     0.0       0.0  0.0 
 0.08  0.0     0.0       0.0  0.16

In [14]:
A_inv3 = AA_inv * A'

5×4 Array{Float64,2}:
 0.2  0.0       0.0  0.0 
 0.0  0.0       0.0  0.25
 0.0  0.333333  0.0  0.0 
 0.0  0.0       0.0  0.0 
 0.4  0.0       0.0  0.0 